In [1]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, initialize_env
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import pytz

In [ ]:
dwh_query("Egypt", """
with thresholds as (
select  day_of_week, 
		hour, 
		half_hour_label, 
		min(order_count) min_count, 
		max(order_count) max_count, 
		median(order_count) as median, 
		min(case when percentile >= 0.1 then order_count end) as lowest_decile, 
		min(case when percentile >= 0.05 then order_count end) as lowest_fifth
from (
select *, 
		percent_rank() over(partition by day_of_week, half_hour_label order by order_count) percentile
from (
select dayname(so.created_at) as day_of_week, 
	   hour(so.created_at) as hour, 
	   so.created_at::date as so_date, 
	   TO_CHAR( DATEADD(minute,CEIL(DATE_PART(minute, created_at) / 30.0) * 30,DATE_TRUNC(hour, created_at)),'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where so.created_at::date between current_date - 60 and current_date 
group by all)) 
group by all ), 

so as (
select dayname(so.created_at) as day_of_week, 
	   hour(so.created_at) as hour, 
	   so.created_at::date as so_date, 
	   TO_CHAR( DATEADD(minute,CEIL(DATE_PART(minute, created_at) / 30.0) * 30,DATE_TRUNC(hour, created_at)),'HH24:MI') AS half_hour_label, 
count(distinct parent_sales_order_id) as order_count
from  public.sales_orders so 
where so.created_at::date = current_date 
group by all) 

select t.*, 
		order_count
from thresholds t 
left join so on t.day_of_week = so.day_of_week and t.half_hour_label = so.half_hour_label
where t.day_of_week = dayname(current_date)
order by t.hour, so.half_hour_label
""", "read", columns=[], conn=None)